In [1]:
import os
import shutil
import subprocess
import gradio as gr
from demucs.apply import apply_model
import torch
import torchaudio

def reencode_audio(input_file, output_file):
    """Reencode audio file using ffmpeg."""
    command = ["ffmpeg", "-i", input_file, "-ar", "44100", "-ac", "2", output_file]
    subprocess.run(command, check=True)

def split_audio(input_file):
    """Split audio into stems using Demucs."""
    # Load Demucs pretrained model
    model = pretrained.get_model("htdemucs")
    model.cpu()
    output_dir = "demucs_output"
    
    # Ensure the output directory exists
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Load and process the audio file
    wav, sr = torchaudio.load(input_file)
    
    # Convert to stereo if mono
    if wav.shape[0] == 1:
        wav = wav.repeat(2, 1)
    
    # Ensure correct sample rate
    if sr != model.samplerate:
        wav = torchaudio.transforms.Resample(sr, model.samplerate)(wav)
    
    # Normalize audio
    wav = wav / wav.abs().max()
    
    # Apply the model
    with torch.no_grad():
        sources = apply_model(model, wav[None], device="cpu")[0]
    
    # Process and save each stem
    stems = ["vocals", "drums", "bass", "other"]
    output_files = {}
    
    for source, stem in zip(sources, stems):
        # Denormalize
        source = source * wav.abs().max()
        
        stem_path = os.path.join(output_dir, f"{stem}.wav")
        torchaudio.save(stem_path, source.cpu(), model.samplerate)
        
        # Reencode the stem
        reencoded_path = os.path.join(output_dir, f"{stem}_reencoded.wav")
        reencode_audio(stem_path, reencoded_path)
        output_files[stem] = reencoded_path
    
    return [output_files.get(stem) for stem in stems]

def gradio_splitter(input_file):
    """Gradio interface function."""
    if input_file is None:
        return [None] * 4
    return split_audio(input_file)

# Create Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🎶 Audio Stem Splitter")
    gr.Markdown("Upload an audio file, and the app will split it into 4 stems: vocals, drums, bass, and other.")
    
    with gr.Row():
        input_file = gr.Audio(type="filepath", label="Upload your audio file")
    
    with gr.Row():
        output_vocals = gr.Audio(label="Vocals")
        output_drums = gr.Audio(label="Drums")
        output_bass = gr.Audio(label="Bass")
        output_other = gr.Audio(label="Other")
    
    submit_button = gr.Button("Split Audio")
    submit_button.click(
        fn=gradio_splitter,
        inputs=[input_file],
        outputs=[output_vocals, output_drums, output_bass, output_other]
    )

if __name__ == "__main__":
    app.launch(debug=True)

/Users/malorenaudin/Library/Caches/pypoetry/virtualenvs/stems_kernel-OtdW1qu4-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
